2D Omnipose segmentation
============================

Before running this notebook, install the latest version of Omnipose from GitHub. This automatically installs our Cellpose fork. 

In [1]:
# make local editable packages automatically reload
%load_ext autoreload
%autoreload 2

In [2]:
# Import dependencies
import numpy as np
from cellpose_omni import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? {}'.format(use_GPU))

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

2023-11-30 13:22:23,719 [INFO] TORCH GPU version not installed/working.
>>> GPU activated? False


C:\Users\Giona\anaconda3\envs\omnipose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
from pathlib import Path
import os
from cellpose_omni import io
import omnipose
import imageio

# Set the path to the desired folder
#basedir = 'CellA_ShME480'

# Get the list of image files from the folder
#files = io.get_image_files(basedir)

# Now that we're in the correct directory, we can use the relative path
video_path = 'CellA_ShME480.avi'

# Create a reader to read the video file
reader = imageio.get_reader(video_path, 'ffmpeg')

# Read all frames in a list
frames = [im for im in reader]

# Prepare file paths for saving the segmentation masks
mask_dir = Path('C:/Users/Giona/ME-480/Progetto 2/masks')  # Directory to save masks
if not mask_dir.exists():
    mask_dir.mkdir(parents=True, exist_ok=True)

# Generate file paths for each frame
files = [mask_dir / f"frame_{i:04d}.tif" for i in range(len(frames))]

In [44]:
from cellpose_omni import io, transforms
from omnipose.utils import normalize99
import numpy as np
import cv2

# Assuming 'frames' contains the frames extracted from the video file
imgs = []
for frame in frames:
    # Convert BGR to grayscale if your frames are in color
    # If your frames are already grayscale, you can skip this conversion
    if len(frame.shape) == 3:  # Check if the frame is colored
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        frame_gray = frame  # If the frame is already in grayscale

    # Normalize and preprocess the frame
    frame_processed = normalize99(frame_gray)
    #frame_processed = np.pad(frame_processed, 10, 'constant', constant_values=0)  # Pad with zeros

    imgs.append(frame_processed)

nimg = len(imgs)
print('\nnumber of images:', nimg)


number of images: 4852


## Initialize model
Here we use one of the built-in model names. We will choose the `bact_phase_omni` model. 

In [151]:
from cellpose_omni import models
from cellpose_omni.models import MODEL_NAMES

model_name = 'cyto2'
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

['bact_phase_omni', 'bact_fluor_omni', 'worm_omni', 'worm_bact_omni', 'worm_high_res_omni', 'cyto2_omni', 'plant_omni', 'bact_phase_cp', 'bact_fluor_cp', 'plant_cp', 'worm_cp', 'cyto', 'nuclei', 'cyto2']
2023-11-30 15:39:52,604 [INFO] >>cyto2<< model set to be used
2023-11-30 15:39:52,605 [INFO] >>>> using CPU


## Run segmentation 

In [ ]:
import time
from tqdm import tqdm
chans = [0,0] #this means segment based on first channel, no second channel 

#n = [1,2,3,4,5,6,7,8,9] # make a list of integers to select which images you want to segment
n = range(nimg) # or just segment them all 

# Assuming you have calculated the mean diameter D of your cells
#mean_diameter = 80  # Replace D with your calculated mean diameter

# Rescaling factor for cyto2_omni
#rescaling_factor = 30.0 / mean_diameter

# define parameters
params = {'channels':chans, # always define this with the model
          'rescale': None, #rescaling_factor, # upscale or downscale your images, None = no rescaling 
          'mask_threshold': 2, # erode or dilate masks with higher or lower values 
          'flow_threshold': 0.45, # default is .4, but only needed if there are spurious masks to clean up; slows down output
          'transparency': True, # transparency in flow output
          'omni': True, # we can turn off Omnipose mask reconstruction, not advised 
          'cluster': True, # use DBSCAN clustering
          'resample': True, # whether or not to run dynamics on rescaled grid or original grid 
          #'verbose': False, # turn on if you want to see more output 
          'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 
          'niter': None, # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
          'augment': False, # Can optionally rotate the image and average outputs, usually not needed 
          'affinity_seg': True, # new feature, stay tuned...
          'diameter': 50

}

total_segmentation_time = 0

# Loop through each image, segment and save
for i in tqdm(n, 'Segmentation progress:'):
    tic = time.time()  # start time for this iteration
    
    # Perform segmentation on the i-th image
    mask, flow, style = model.eval(imgs[i], **params)
    
    # Calculate segmentation time for this image
    seg_time = time.time() - tic
    total_segmentation_time += seg_time  # add to total time
    
    print(f'Segmentation time for image {i}: {seg_time}s')
    
    # Define the file path for saving the results
    file_path = files[i]  # Assuming 'files' is your list of file paths for the images
    
    # Save the masks for the i-th image
    io.save_masks([imgs[i]], [mask], [flow], [file_path], 
                  png=True,
                  tif=True,
                  suffix='',  # Add suffix if needed
                  save_flows=False,
                  save_outlines=True,
                  dir_above=0,
                  in_folders=True,
                  save_txt=True,
                  save_ncolor=False)


print(f'Total segmentation time for all images: {total_segmentation_time}s')

average_time_per_image = total_segmentation_time / nimg
expected_total_time = average_time_per_image * nimg
print(f'Expected total time for segmenting all images: {expected_total_time}s')

Segmentation progress::   0%|                                                                 | 0/4852 [00:00<?, ?it/s]

Segmentation time for image 0: 0.9770810604095459s


Segmentation progress::   0%|                                                       | 1/4852 [00:01<2:13:49,  1.66s/it]

Segmentation time for image 1: 0.9999620914459229s


Segmentation progress::   0%|                                                       | 2/4852 [00:03<2:15:25,  1.68s/it]

...

Segmentation time for image 377: 0.894995927810669s


Segmentation progress::   8%|████▏                                                | 378/4852 [10:31<1:56:10,  1.56s/it]

Segmentation time for image 378: 0.8915908336639404s


Segmentation progress::   8%|████▏                                                | 379/4852 [10:32<1:55:17,  1.55s/it]

Segmentation time for image 379: 0.9267306327819824s
